In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd

/home/sixigma/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sixigma/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sixigma/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sixigma/.local/lib/python3.6/site-packages/tensorboard/compat

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

In [3]:
data = pd.read_csv('DSL-StrongPasswordData.csv')

time_stamp = data.iloc[:, 3:]
# https://github.com/pytorch/pytorch/issues/2267#issuecomment-447923931
np_time_stamp = np.array(time_stamp, dtype=np.float32)

typist = np.zeros(data.shape[0])
typist_list = list(set(data.subject))
for i, s in data.subject.iteritems():
    typist[i] = typist_list.index(s)

In [4]:
class KeyStrokeDataset(torch.utils.data.Dataset):
    """
    A Map-style datasets.
    """
    def __init__(self, time_stamp, typist, transform=None):
        self.x = time_stamp
        self.label = typist
        self.transform = transform

    def __len__(self):
        # for sampler
        return len(self.label)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        _x = self.x[idx, :]
        _label = self.label[idx]
        sample = {'x': _x, 'label': _label}

        if self.transform:
            sample = self.transform(sample)

        return sample
    
    def __iter__(self):
        """
        For a Iterable-style datasets.
        """
        raise NotImplementedError

In [5]:
training = KeyStrokeDataset(np_time_stamp, typist)

In [6]:
sampler = torch.utils.data.RandomSampler(training)
batch_sampler = torch.utils.data.BatchSampler(sampler, batch_size=192, drop_last=True)

In [7]:
data_loader = torch.utils.data.DataLoader(
        training, batch_sampler=batch_sampler, num_workers=4,
        collate_fn=None, pin_memory=True)

In [8]:
for d in data_loader:
    print(d)
    break

{'x': tensor([[0.0747, 0.1746, 0.0999,  ..., 0.2746, 0.2147, 0.0631],
        [0.0881, 0.1468, 0.0587,  ..., 0.2273, 0.1149, 0.0873],
        [0.0937, 0.2509, 0.1572,  ..., 0.2723, 0.1923, 0.0803],
        ...,
        [0.0831, 0.1081, 0.0250,  ..., 0.1624, 0.0703, 0.1346],
        [0.0964, 0.1230, 0.0266,  ..., 0.2198, 0.1461, 0.0576],
        [0.0757, 0.3681, 0.2924,  ..., 0.8356, 0.7683, 0.0718]]), 'label': tensor([26., 27., 29., 33., 14., 39., 29.,  1.,  0., 41., 20., 45., 25.,  9.,
        19., 11., 35., 12., 25., 50.,  7., 50., 41., 28., 28., 10., 46., 29.,
        47., 28., 13., 12.,  0., 49., 40.,  7., 49., 10., 27.,  3., 19.,  6.,
        37., 31., 27., 44.,  4., 42., 41., 40., 28., 31., 33.,  1., 39., 26.,
        35., 32., 16.,  3.,  1., 22., 13.,  6., 48., 47., 50., 39., 34., 23.,
        40.,  7., 10., 16.,  8., 32., 34., 15., 43.,  2.,  8., 22.,  9., 33.,
        32., 27., 37., 21., 49., 29., 30., 29.,  2., 30., 25., 45., 49., 27.,
        36., 44., 16.,  7., 42., 32.,  3

In [9]:
class LinearModel(nn.Module):
    def __init__(self, num_feature, hidden_dim, num_classes):
        super(LinearModel, self).__init__()
        self.input_layer = nn.Linear(num_feature, hidden_dim)

        self.hidden2class = nn.Linear(hidden_dim, num_classes)
        
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, time_stamp):
        """
        Args:
            time_stamp: (num_feature, batch_size, num_channels)
        Output:
            pred_class: (batch_size, num_classes)
        """
        h = self.input_layer(time_stamp.view(len(time_stamp), -1))
        metric = self.hidden2class(h)
        
        return self.softmax(metric)

In [11]:
model = LinearModel(31, 50, num_layers=3, num_classes=51).to(device)

In [12]:
# https://github.com/pytorch/pytorch/issues/2267#issuecomment-447923931
# RuntimeError: cuDNN error: CUDNN_STATUS_BAD_PARAM
# data should be float32
with torch.no_grad():
    for d in data_loader:
        x = d['x'].to(device)
        pred = model(x)
        print(torch.argmax(pred, 1))
        break

tensor([ 3,  3, 10, 44,  2, 10, 44, 44, 10,  9, 10, 44,  2, 44, 44, 10,  3, 44,
        44, 44, 44, 44, 44, 36, 44, 10, 44, 44, 10,  3, 44, 44, 10, 10,  2, 44,
         1, 10, 44, 10, 35,  2, 44, 10, 10, 44,  3, 10, 44, 44, 10,  3, 10, 44,
        10,  3, 10, 44,  3, 10, 10, 44, 44, 44, 10,  3, 10, 10,  3, 10, 10, 35,
        44, 35, 46,  3, 10, 10, 44, 10, 10,  3,  3, 10,  3, 44, 10,  3, 10, 10,
        10, 44, 44, 35,  3, 44, 44, 10, 44, 10, 10, 44, 44, 44, 44, 10, 10,  2,
        44, 10, 10, 10, 44, 10, 10, 44, 10, 44, 44, 44, 10, 10, 10, 10,  2,  2,
        44, 10,  3,  3, 10, 36,  3, 35, 10, 44, 44, 10, 10, 44, 44, 44, 44, 35,
        35, 10, 10, 10, 10, 10, 44, 10, 10, 10,  3, 44, 44,  2, 44, 44,  2, 10,
         2, 10, 10, 44, 44, 10, 44,  3, 10, 10, 44, 10, 44, 30, 44, 44, 44, 10,
        10, 44, 10, 10, 44, 44, 44, 10, 44, 44, 44,  6], device='cuda:0')


## Simple Training

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(lstm.parameters(), lr=0.01, momentum=0.9)

In [19]:
epoch = 5
for e in range(epoch):
    for d in data_loader:
        x = d['x'].to(device)
        labels = d['label'].to(device)
        
        optimizer.zero_grad()
        pred = model(x)
        
        loss = loss_fn(pred, labels.long())
        
        loss.backward()
        optimizer.step()

In [25]:
with torch.no_grad():
    for d in data_loader:
        x = d['x'].to(device)
        pred = model(x)
        print(torch.argmax(pred, 1), d['label'])
        break

tensor([34, 42, 34, 19, 29, 34, 34,  6, 29,  0, 29,  6, 19, 42,  4,  6, 29, 29,
        34,  4,  4, 29, 29, 19,  6, 34, 34, 29,  0,  6, 29, 34,  6, 29, 34, 34,
        19,  6, 29, 29, 19,  4,  0,  4, 29, 42, 34, 34,  4, 34,  6,  4,  6,  0,
         6, 19, 29, 29, 29, 34, 34, 29, 34,  0, 34,  4, 34,  4,  4, 34, 19, 29,
        29, 29,  4,  6, 29, 29, 34,  6, 29,  6,  4,  0, 29, 29,  6, 34, 34,  0,
        19, 29, 29, 34,  4,  4,  6, 34, 29, 29, 29, 34, 34, 19, 34,  6, 19,  0,
        29,  6, 34, 34, 29, 29, 34,  4, 34, 29, 34, 29,  0, 19,  4, 29, 34, 34,
        29,  4, 34,  0, 34, 34, 34, 29, 34, 29, 34, 34, 19, 29,  4, 19, 34, 34,
        34,  6, 29,  6,  4, 29, 29, 29, 19, 19, 29,  4, 19, 34, 29, 19, 29, 29,
        34, 29,  4,  6, 19, 34,  0,  4, 29, 19, 29,  4, 34, 19, 34, 19, 29,  6,
        29, 34, 29,  6, 34, 29, 29, 34, 29, 19, 29,  0], device='cuda:0') tensor([15., 46., 31., 50., 31.,  5., 34., 37.,  2.,  0., 49.,  6., 19., 42.,
        40.,  6., 22., 28., 25., 11., 11., 40., 

## Tensorboard

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
writer = SummaryWriter('runs/')
_step = 0
epoch = 5
data_loader_len = len(data_loader)
for e in range(epoch):
    for d in data_loader:
        x = d['x'].to(device)
        labels = d['label'].to(device)
        
        optimizer.zero_grad()
        pred = model(x)
        
        loss = loss_fn(pred, labels.long())
        
        loss.backward()
        optimizer.step()
        
        _step += 1
        if _step % 100 == 0:
            writer.add_scalar("Loss/train", loss, e * data_loader_len + _step)

    for i, p in enumerate(model.parameters()):
        writer.add_histogram("parameter_{}".format(i), p.data, e * data_loader_len + _step)
        
writer.flush()
writer.close()

## Learning Rate Decay

In [ ]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100,
                                                       eta_min=0.0005)

In [ ]:
writer = SummaryWriter('runs/')
_step = 0
epoch = 5
data_loader_len = len(data_loader)
for e in range(epoch):
    for d in data_loader:
        x = d['x'].to(device)
        labels = d['label'].to(device)
        
        optimizer.zero_grad()
        pred = model(x)
        
        loss = loss_fn(pred, labels.long())
        
        loss.backward()
        optimizer.step()
        
        _step += 1
        if _step % 100 == 0:
            writer.add_scalar("Loss/train", loss, e * data_loader_len + _step)
            writer.add_scalar("Loss/lr", scheduler.get_last_lr()[0], e * data_loader_len + _step)

    for i, p in enumerate(model.parameters()):
        writer.add_histogram("parameter_{}".format(i), p.data, e * data_loader_len + _step)
            
    scheduler.step()

writer.flush()
writer.close()

## Save and Load

In [ ]:
writer = SummaryWriter('runs/')
_step = 0
epoch = 5
data_loader_len = len(data_loader)
for e in range(epoch):
    for d in data_loader:
        x = d['x'].to(device)
        labels = d['label'].to(device)
        
        optimizer.zero_grad()
        pred = model(x)
        
        loss = loss_fn(pred, labels.long())
        
        loss.backward()
        optimizer.step()
        
        _step += 1
        if _step % 100 == 0:
            writer.add_scalar("Loss/train", loss, e * data_loader_len + _step)
            writer.add_scalar("Loss/lr", scheduler.get_last_lr()[0], e * data_loader_len + _step)

    for i, p in enumerate(model.parameters()):
        writer.add_histogram("parameter_{}".format(i), p.data, e * data_loader_len + _step)
            
    scheduler.step()
    
    torch.save(model.state_dict(), 'model/linear-{}.pt'.format(e))

writer.flush()
writer.close()